BADM 554 Group 3 Mini Project 1 Analysis Queries

Final Business Recommendation

Based on our multi-stage analysis, Philadelphia, PA is the strongest market to enter for our new restaurant concept. Among our candidate cities, Philadelphia shows the largest dining demand, with the highest number of check-ins and a strong concentration in the evening period, indicating a vibrant dinner economy. In addition, 47.88% of restaurants in Philadelphia provide parking, showing that accessibility is an important factor for customers—offering parking will be a key competitive requirement.

Price range segmentation shows that higher-priced restaurants (Price Range 3–4) have the highest survival rates in our target cities, suggesting consumers are willing to pay for quality. Meanwhile, cuisine analysis reveals that American and Mexican cuisines dominate the market, leaving room for differentiation by introducing non-American, non-Mexican dining concepts. Although Philadelphia is predominantly casual dining, the very limited supply of elevated experiences suggests an opportunity to position our restaurant in the premium-casual to dressy category—stylish, but approachable in price and atmosphere.

Strategic Positioning Summary

Location: Philadelphia, PA

Price Point: Mid-to-high range (PR 3–4)

Service Focus: Dinner-oriented, with strong emphasis on Sunday and Monday, when competition is lower

Experience: Premium-casual atmosphere suitable for social gatherings and celebrations

Accessibility: On-site or partnered parking required

Food Strategy: Differentiate with unique cuisine instead of competing with dominant American/Mexican supply


With this positioning, we aim to maximize traffic during Philadelphia’s most active dining periods, while capturing unmet demand in higher-end dining and better accessibility.

How We Reached This Conclusion

We first identified top restaurant markets using check-ins, then filtered to the strongest restaurant economies: Philadelphia, Tampa, and Reno. We compared restaurant count, business survival rate, price range, attire style, dining time behavior, and parking availability across these cities. Out of all candidates, Philadelphia consistently ranked high in:

Market size

Dinner demand concentration

Price-range survival advantage


While it showed slightly lower business survival rate than Tucson or Reno, this is offset by its significantly larger market, meaning a higher potential return.

Thus, our strategy aligns with:

High demand (evening concentration)

Underserved premium dining supply

Higher-priced category profitability

Consumer requirement for parking

In [ ]:
import os
os.listdir()
!pip install pymssql pandas
!pip install sqlalchemy pyodbc tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 6.6 MB/s eta 0:00:00


Link to data server

In [ ]:
import pymssql

#The database login information has been removed for security purposes

server = ''
database = ''
username = ''
password = ''

conn = pymssql.connect(server=server, user=username, password=password, database=database)
cursor = conn.cursor()
print("✅ Reconnected to Azure SQL!")
cursor.execute("SELECT name FROM sys.objects WHERE type='U'")
for row in cursor.fetchall():
    print(row)

✅ Reconnected to Azure SQL!
('tb_business',)
('tb_category',)
('tb_attribute',)
('tb_checkin',)
('tb_elite',)
('tb_tip',)
('tb_user',)
('tb_review',)


1. Which area has the most check-ins?

In [ ]:
import pandas as pd

query = """
SELECT
    b.state,
    b.city,
    COUNT(*) AS total_checkins
FROM tb_checkin c
JOIN tb_business b
    ON c.business_id = b.business_id
GROUP BY b.state, b.city
ORDER BY total_checkins DESC;
"""

cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=['state', 'city', 'total_checkins'])
print(df.head(10))   # view top 10

cursor.close()

  state          city  total_checkins
0    PA  philadelphia         1836650
1    LA    neworleans         1486543
2    FL         tampa          919654
3    IN  indianapolis          886068
4    TN     nashville          818144
5    AZ        tucson          772639
6    NV          reno          712453
7    MO    saintlouis          639084
8    CA  santabarbara          507917
9    AB      edmonton          193775


Based on the analysis above, the selected areas represent the top 10 cities with the highest check-in volumes, indicating high dining activity, strong customer engagement, and potentially larger market size.
Therefore, subsequent analyses will focus on these ten cities to identify the most suitable restaurant location and market strategy.

2. Which city has the most restaurants?

In [ ]:
query = """
SELECT
    b.city,
    b.state,
    COUNT(*) AS restaurant_count
FROM tb_business b
JOIN tb_category c
    ON b.business_id = c.business_id
WHERE c.categories LIKE '%Restaurants%'
  AND b.city IN ('philadelphia', 'neworleans', 'tampa', 'indianapolis', 'nashville',
                 'tucson', 'reno', 'saintlouis', 'santabarbara', 'edmonton')
GROUP BY b.city, b.state
ORDER BY restaurant_count DESC;
"""

cursor.execute(query)
rows = cursor.fetchall()

# ✅ format result into DataFrame
df_result = pd.DataFrame(rows, columns=["City", "State", "Restaurant_Count"])
df_result


,City,State,Restaurant_Count
0,philadelphia,PA,5868
1,tampa,FL,2971
2,indianapolis,IN,2866
3,nashville,TN,2515
4,tucson,AZ,2469
5,neworleans,LA,2278
6,edmonton,AB,2169
7,saintlouis,MO,1791
8,reno,NV,1289
9,santabarbara,CA,776


3. Which area has the highest proportion of open restaurants?

In [ ]:
query = """
SELECT
    b.city,
    COUNT(*) AS total_businesses,
    SUM(CASE WHEN b.is_open = 1 THEN 1 ELSE 0 END) AS open_businesses,
    ROUND(
        CAST(SUM(CASE WHEN b.is_open = 1 THEN 1 ELSE 0 END) AS FLOAT) /
        NULLIF(CAST(COUNT(*) AS FLOAT), 0)
    , 4) AS open_rate
FROM tb_business b
WHERE b.city IN ('philadelphia', 'neworleans', 'tampa', 'indianapolis', 'nashville',
                 'tucson', 'reno', 'saintlouis', 'santabarbara', 'edmonton')
GROUP BY b.city
HAVING COUNT(*) >= 20
ORDER BY open_rate DESC, total_businesses DESC;
"""


cursor.execute(query)
rows = cursor.fetchall()


import pandas as pd
df = pd.DataFrame(rows, columns=["City", "Total_Businesses", "Open_Businesses", "Open_Rate"])
df


,City,Total_Businesses,Open_Businesses,Open_Rate
0,tucson,9262,7544,0.8145
1,reno,5940,4767,0.8025
2,tampa,9069,7237,0.7980
3,santabarbara,3837,3028,0.7892
4,indianapolis,7545,5899,0.7818
5,edmonton,5056,3918,0.7749
6,nashville,6981,5408,0.7747
7,neworleans,6216,4655,0.7489
8,philadelphia,14577,10549,0.7237
9,saintlouis,4833,3409,0.7054


Among them, ***Philadelphia*** has the largest restaurant market, while ***Tucson, Reno, and Santa Barbara*** show healthier business survival rates with a higher percentage of open establishments. Cities such as ***Tampa, Indianapolis, and Nashville*** maintain a strong balance between restaurant volume and stability, making them attractive expansion targets. Going forward, our restaurant opportunity analysis will focus on these 7 cities to better understand market conditions, customer behavior, and competitive potential.

4. Which type of restaurant is the most common?

In [ ]:
query = """
SELECT
   b.city,
   b.state,
   COUNT(DISTINCT CASE WHEN c.categories LIKE '%American%' THEN b.business_id END) AS american_restaurants,
   COUNT(DISTINCT CASE WHEN c.categories LIKE '%Mexican%' THEN b.business_id END) AS mexican_restaurants,
   COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END) AS total_restaurants,
   CASE
       WHEN COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END) = 0
           THEN NULL
       ELSE CAST(COUNT(DISTINCT CASE WHEN c.categories LIKE '%American%' THEN b.business_id END) AS FLOAT)
            / COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END)
   END AS american_ratio,
   CASE
       WHEN COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END) = 0
           THEN NULL
       ELSE CAST(COUNT(DISTINCT CASE WHEN c.categories LIKE '%Mexican%' THEN b.business_id END) AS FLOAT)
            / COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END)
   END AS mexican_ratio
FROM tb_business b
JOIN tb_category c
   ON b.business_id = c.business_id
WHERE b.city IN ('philadelphia', 'tampa', 'indianapolis', 'nashville',
                'tucson', 'reno','santabarbara')
GROUP BY b.city, b.state
HAVING COUNT(DISTINCT CASE WHEN c.categories LIKE '%Restaurant%' THEN b.business_id END) > 0
ORDER BY mexican_ratio DESC;
"""


cursor.execute(query)
rows = cursor.fetchall()


df_result = pd.DataFrame(rows, columns=[
   "City", "State", "American_Restaurants", "Mexican_Restaurants",
   "Total_Restaurants", "American_Ratio", "Mexican_Ratio"
])
df_result


,City,State,American_Restaurants,Mexican_Restaurants,Total_Restaurants,American_Ratio,Mexican_Ratio
0,tucson,AZ,560,512,2473,0.226446,0.207036
1,santabarbara,CA,222,119,768,0.289062,0.154948
2,reno,NV,308,196,1290,0.238760,0.151938
3,indianapolis,IN,808,380,2863,0.282221,0.132728
4,nashville,TN,741,272,2506,0.295690,0.108540
5,tampa,FL,823,231,2968,0.277291,0.077830
6,philadelphia,PA,1308,316,5857,0.223323,0.053953


We identified five cities—**Philadelphia (PA), Tucson (AZ), Tampa (FL), Indianapolis (IN), and Reno (NV)**—where American and Mexican restaurants represent a relatively small portion of the market. These locations show more diverse dining competition and offer better opportunities for a restaurant concept that differentiates from the dominant cuisines in other cities. Our following analyses will focus on these selected cities to guide strategic decisions.

5. What kind of dining style does the restaurant have?

In [ ]:
query = """
SELECT
   b.city,
   b.state,
   COUNT(DISTINCT a.business_id) AS total_with_attire,
   COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%casual%' THEN a.business_id END) AS casual_count,
   COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%dressy%' THEN a.business_id END) AS dressy_count,
   COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%formal%' THEN a.business_id END) AS formal_count,
   CAST(COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%casual%' THEN a.business_id END) AS FLOAT)
       / NULLIF(COUNT(DISTINCT a.business_id), 0) AS casual_ratio,
   CAST(COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%dressy%' THEN a.business_id END) AS FLOAT)
       / NULLIF(COUNT(DISTINCT a.business_id), 0) AS dressy_ratio,
   CAST(COUNT(DISTINCT CASE WHEN LOWER(a.attribute_value) LIKE '%formal%' THEN a.business_id END) AS FLOAT)
       / NULLIF(COUNT(DISTINCT a.business_id), 0) AS formal_ratio
FROM tb_business b
JOIN tb_attribute a
   ON b.business_id = a.business_id
JOIN tb_category c
   ON b.business_id = c.business_id
WHERE a.attribute_name = 'RestaurantsAttire'
 AND c.categories LIKE '%Restaurant%'
 AND b.city IN ('philadelphia', 'tampa', 'indianapolis',
                'tucson', 'reno')
GROUP BY b.city, b.state
ORDER BY casual_ratio DESC;
"""


cursor.execute(query)
rows = cursor.fetchall()


df_result = pd.DataFrame(rows, columns=[
   "City", "State", "Total_With_Attire",
   "Casual_Count", "Dressy_Count", "Formal_Count",
   "Casual_Ratio", "Dressy_Ratio", "Formal_Ratio"
])
df_result


,City,State,Total_With_Attire,Casual_Count,Dressy_Count,Formal_Count,Casual_Ratio,Dressy_Ratio,Formal_Ratio
0,tucson,AZ,1902,1880,20,2,0.988433,0.010515,0.001052
1,reno,NV,1040,1020,19,1,0.980769,0.018269,0.000962
2,indianapolis,IN,2165,2109,43,6,0.974134,0.019861,0.002771
3,philadelphia,PA,4353,4231,112,9,0.971973,0.025729,0.002068
4,tampa,FL,2105,2041,53,6,0.969596,0.025178,0.002850


6. Aveareg Price Ranking In the Areas

In [ ]:
import pandas as pd


query = """
WITH price AS (
    SELECT
        b.state,
        b.city,
        TRY_CAST(a.attribute_value AS DECIMAL(9,4)) AS price
    FROM tb_attribute a
    JOIN tb_business b
        ON a.business_id = b.business_id
    WHERE a.attribute_name = 'RestaurantsPriceRange2'
      AND a.attribute_value IN ('1','2','3','4')
      AND (
          (b.state = 'PA' AND b.city = 'philadelphia') OR
          (b.state = 'FL' AND b.city = 'tampa') OR
          (b.state = 'IN' AND b.city = 'indianapolis') OR
          (b.state = 'AZ' AND b.city = 'tucson') OR
          (b.state = 'NV' AND b.city = 'reno')
      )
)
SELECT
    state,
    city,
    CAST(AVG(price) AS DECIMAL(9,4)) AS avg_price_range,
    COUNT(*) AS sample_size
FROM price
GROUP BY state, city
ORDER BY avg_price_range DESC;
"""


cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
cursor.close()


df_price = pd.DataFrame(rows, columns=['state','city','avg_price_range','sample_size'])
print(df_price)


  state          city avg_price_range  sample_size
0    NV          reno          1.8154         2827
1    PA  philadelphia          1.7559         8903
2    FL         tampa          1.7551         4802
3    AZ        tucson          1.7227         4673
4    IN  indianapolis          1.6961         4511


Based on our price range analysis, the top three cities with the highest average restaurant pricing are ***Reno (NV), Philadelphia (PA), and Tampa (FL)***, suggesting stronger consumer willingness to spend on dining. However, attire data shows that restaurants in these cities are overwhelmingly casual, with more than 96% categorized as casual dining. This indicates that although customers in these markets are willing to pay more, the supply of upscale dining experiences remains limited. Therefore, these cities present a strong opportunity for introducing a more premium, dressy restaurant concept to fill the gap in the market.

7. Princing Position Verify

In [ ]:
query = """
SELECT
    CASE
        WHEN a.attribute_value IN ('1','2','3','4') THEN a.attribute_value
        ELSE 'Unknown'
    END AS price_range,
    COUNT(*) AS total_businesses,
    SUM(CASE WHEN b.is_open = 1 THEN 1 ELSE 0 END) AS open_businesses,
    ROUND(
        1.0 * SUM(CASE WHEN b.is_open = 1 THEN 1 ELSE 0 END) / COUNT(*),
    4) AS open_rate
FROM tb_attribute a
JOIN tb_business b
    ON a.business_id = b.business_id
WHERE a.attribute_name = 'RestaurantsPriceRange2'
  AND (
        (b.city = 'reno' AND b.state = 'NV') OR
        (b.city = 'philadelphia' AND b.state = 'PA') OR
        (b.city = 'tampa' AND b.state = 'FL')
      )
GROUP BY CASE
            WHEN a.attribute_value IN ('1','2','3','4') THEN a.attribute_value
            ELSE 'Unknown'
         END
ORDER BY price_range;
"""

cursor.execute(query)
rows = cursor.fetchall()
rows


[('1', 5674, 3758, Decimal('0.662300000000')),
 ('2', 9292, 6238, Decimal('0.671300000000')),
 ('3', 1329, 850, Decimal('0.639600000000')),
 ('4', 237, 165, Decimal('0.696200000000')),
 ('Unknown', 4, 2, Decimal('0.500000000000'))]

In our three selected cities, higher-priced restaurants (Price Range 4) show the highest survival rate, indicating a stronger market support for premium dining concepts.

8. What are the most common check-in times? (ex. lunch special)

In [ ]:
import pandas as pd
import sqlalchemy

#The database login information has been removed for security purposes

engine = sqlalchemy.create_engine(
    ""
)




query = """
SELECT b.state, b.city, c.date
FROM tb_checkin c
JOIN tb_business b
    ON c.business_id = b.business_id
WHERE
          (b.state = 'PA' AND b.city = 'philadelphia') OR
          (b.state = 'FL' AND b.city = 'tampa') OR
          (b.state = 'NV' AND b.city = 'reno')
"""


df = pd.read_sql(query, engine)


df['date'] = pd.to_datetime(df['date'])
df['hour'] = df['date'].dt.hour


def get_time_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 15:
        return 'Noon'
    elif 15 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 23:
        return 'Evening'
    else:
        return 'Night'


df['time_period'] = df['hour'].apply(get_time_period)


pivot_df = df.pivot_table(
    index=['state', 'city'],
    columns='time_period',
    values='date',
    aggfunc='count',
    fill_value=0
).reset_index()




pivot_df = pivot_df[['state', 'city', 'Morning', 'Noon', 'Afternoon', 'Evening']]
pivot_df['total_checkins'] = pivot_df[['Morning','Noon','Afternoon','Evening']].sum(axis=1)
pivot_df = pivot_df.sort_values(by='total_checkins', ascending=False)


print(pivot_df.head(20))

time_period state          city  Morning    Noon  Afternoon  Evening  \
2              PA  philadelphia    56007  153908     374026   644080   
0              FL         tampa    32397   79118     208015   319848   
1              NV          reno    30580   20340      79453   258638   

time_period  total_checkins  
2                   1228021  
0                    639378  
1                    389011  


***Philadelphia*** stands out as the strongest market opportunity with nearly double the total check-ins of Tampa and more than three times those of Reno. Additionally, customer activity in ***Philadelphia*** is highly concentrated in the evening hours, while morning and noon check-ins remain relatively low. This suggests that a restaurant concept focused on dinner-oriented dining would better align with local demand patterns.

9. Operating Hour in ***Philadelphia***

In [ ]:
query = """
SELECT
    'Monday' AS day,
    COUNT(*) AS open_count
FROM tb_business
WHERE city = 'philadelphia' AND monday_hours <> ''
UNION ALL
SELECT
    'Tuesday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND tuesday_hours <> ''
UNION ALL
SELECT
    'Wednesday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND wednesday_hours <> ''
UNION ALL
SELECT
    'Thursday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND thursday_hours <> ''
UNION ALL
SELECT
    'Friday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND friday_hours <> ''
UNION ALL
SELECT
    'Saturday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND saturday_hours <> ''
UNION ALL
SELECT
    'Sunday',
    COUNT(*)
FROM tb_business
WHERE city = 'philadelphia' AND sunday_hours <> ''
ORDER BY open_count DESC;
"""

cursor.execute(query)
rows = cursor.fetchall()

print("📊 Philadelphia Daily Operating Availability:")
for row in rows:
    print(row)


📊 Philadelphia Daily Operating Availability:
('Thursday', 11563)
('Friday', 11563)
('Wednesday', 11390)
('Tuesday', 10898)
('Saturday', 10488)
('Monday', 10290)
('Sunday', 7982)


Based on the business hours distribution in Philadelphia, ***Sunday and Monday*** have the fewest open restaurants, indicating lower competition during the beginning and end of the week. This suggests a market opportunity—opening on these two days can help capture unmet demand and increase visibility.

10. ***Parking Availability*** in Philadelphia

In [ ]:
query = """
SELECT
    COUNT(*) AS total_restaurants,
    SUM(CASE
            WHEN a.attribute_value LIKE '%True%' THEN 1
            ELSE 0
        END) AS parking_available,
    CAST(SUM(CASE
                WHEN a.attribute_value LIKE '%True%' THEN 1
                ELSE 0
            END) AS FLOAT) / COUNT(*) AS parking_ratio
FROM tb_business b
LEFT JOIN tb_attribute a
    ON b.business_id = a.business_id
    AND a.attribute_name = 'BusinessParking'
WHERE b.city = 'philadelphia' AND b.state = 'PA';
"""

cursor.execute(query)
row = cursor.fetchone()
print(f"Total Restaurants: {row[0]}")
print(f"Parking Available: {row[1]}")
print(f"Parking Ratio: {row[2]:.4f}")


Total Restaurants: 14575
Parking Available: 6978
Parking Ratio: 0.4788


Our analysis shows that 47.88% of restaurants in Philadelphia provide parking, which means nearly half of customers expect this convenience. To stay competitive and improve accessibility, ***our restaurant should include parking availability*** as part of the location strategy.

11. Which place (or city) has the highest average rating?

In [ ]:
query = """
SELECT TOP 10
    b.name,
    b.city,
    b.state,
    AVG(r.stars) AS avg_stars,
    COUNT(*) AS num_reviews
FROM tb_review r
JOIN tb_business b
    ON r.business_id = b.business_id
GROUP BY b.name, b.city, b.state
ORDER BY avg_stars DESC, num_reviews DESC;
"""

df_top = pd.read_sql(query, conn)
print(df_top)


0                Twisted Twig Fine Florals  santabarbara    CA          5   
1                   Carpinteria Lock & Key   carpinteria    CA          5   
2                    Taylor Home Solutions     nashville    TN          5   
3                     Circle City Pet Care  indianapolis    IN          5   
4                     Hidden History Tours    neworleans    LA          5   
5         MICHAEL & ANNA COSTA PHOTOGRAPHY  santabarbara    CA          5   
6  Head & Heart Photography by Kiel Rucker  santabarbara    CA          5   
7                              Barnet Fair  philadelphia    PA          5   
8                          Freedom Massage       malvern    PA          5   
9                            Terrell House    neworleans    LA          5  